# Numba et cuda

<img src="figures/numba_blue_icon_rgb.png" alt="Drawing" style="width: 20%;"/>

<center>**Loic Gouarin**</center>
<center>*6 juillet 2017*</center>

L'une des forces de Numba est de pouvoir utiliser les mêmes principes que précédemment pour pouvoir développer rapidement des kernels pouvant tourner sur des cartes GPU. Nous allons voir dans la suite avec quelle simplicité nous pouvons faire du Python et laisser Numba générer les kernels cuda.

Il est néanmoins nécessaire d'avoir quelques connaissances sur l'architecture d'un GPU afin de tirer partie de toutes les performances possibles. Voici comment est constituée une carte GPU

![cuda](figures/cuda_1.gif)
![cuda](figures/cuda_memory.jpg)

Il est important de noter que vous n'aurez pas toutes les fonctionalités que peut offrir cuda. Par exemple, vous ne pouvez pas pour le moment transférer des données sur la mémoire texture, ni faire du parallélisme dynamique.

Afin d'utiliser toutes les méthodes pour l'utilisation de cuda dans Numba, il suffit d'importer

In [1]:
from numba import cuda

De là, vous avez accès à un jit et à différentes fonctionalités que nous allons décrire dans la suite. 

Prenons un premier exemple qui réalise la somme de deux vecteurs.

In [2]:
@cuda.jit
def somme(a, b, c):
    i = cuda.threadIdx.x + cuda.blockIdx.x*cuda.blockDim.x
    if i < a.size:
        c[i] = a[i] + b[i]    

Numba permet de récupérer les identifiants des threads, des blocs et la dimension des blocs via cuda.threadIdx.x, cuda.blockIdx.x et cuda.blockDim.x ce qui est classique si vous connaissez un peu le langage cuda. Nous travaillons ici en 1d mais nous avons également accès à cuda.threadIdx.y(z), cuda.blockIdx.y(z) et cuda.blockDim.y(z) pour travailler en 2d ou 3d.

Maintenant que nous avons notre kernel, nous devons définir le nombre de threads par bloc et le nombre de blocs qui sont étroitement liés à nos paramètres d'entrée.

In [2]:
import numpy as np
a = np.random.random(1000)
b = np.random.random(1000)
c = np.zeros_like(a)

threadsperblock = 128
blockspergrid = (a.size + (threadsperblock - 1)) // threadsperblock

Il suffit à présent d'appeler le kernel avec ces informations.

In [4]:
somme[blockspergrid, threadsperblock](a, b, c)

Vérifions que le résultat est correct.

In [5]:
np.all(c == a+b)

True

On pourra constater que nous n'avons à aucun moment transféré les données du CPU (appelé host) au GPU (appelé device). Numba s'est chargé de le faire à l'appel de la fonction. De même qu'il a récupéré les données du GPU pour pouvoir ensuite avoir les valeurs de c.

Nous verrons par la suite que nous pouvons gérer nous même la mémoire (copie de données host vers device et device vers host). Ce qui peut-être plus intéressant lorsque l'on cherche à avoir des performances: plus les données restent sur le device et mieux c'est !!

Numba offre la possibilité d'avoir les coordonnées absolues dans la grille par l'opérateur *grid(ndim)*. Nous pourrions réécrire l'exemple précédent par

In [6]:
@cuda.jit
def somme(a, b, c):
    i = cuda.grid(1) # cuda.threadIdx.x + cuda.blockIdx.x*cuda.blockDim.x
    if i < a.size:
        c[i] = a[i] + b[i]

## Gestion de la mémoire

Numba offre différentes fonctions pour gérer la mémoire

- copier un tableau du host vers le device
- copier un tableau du device vers le host
- créer des tableaux directement sur le device

#### Copier un tableau du host vers le device

In [7]:
a = np.zeros(1000)
d_a = cuda.to_device(a)

In [8]:
d_a.shape

(1000,)

In [9]:
d_a.dtype

dtype('float64')

#### Copier un tableau du device vers le host

In [10]:
a = d_a.copy_to_host()

#### Créer un tableau sur le device

Lorsque vous créez un tableau sur le device, celui-ci à sa place allouée mais n'est pas initialisé (il faut le voir comme si vous faisiez un numpy.empty). 

In [11]:
d_a = cuda.device_array(100)

In [12]:
d_b = cuda.device_array_like(b)

Reprenons notre exemple précédent. Pour faire la somme de deux vecteurs, nous devons copier a et b sur le device. En revanche, le résultat c n'est utile que sur le device et peut être ensuite rappatrié sur le host.

Voici ce que ça donne avec une bonne gestion de la mémoire

In [13]:
import numpy as np
a = np.random.random(10000)
b = np.random.random(10000)
d_c = cuda.device_array_like(a)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)

somme[blockspergrid, threadsperblock](d_a, d_b, d_c)

c = d_c.copy_to_host()

Quel est le gain attendu ? Pour cet exemple, il faut juste voir qu'on évite 3 transferts entre le host et le device:

- c n'est pas copier du host vers le device
- a et b ne sont pas récupérés à la fin du device

## Cas du produit matrice-matrice

Nous allons prendre un exemple simple afin d'expliquer un peu mieux la façon d'obtenir de bonnes performances lorsque l'on veut calculer sur une carte graphique.

Nous souhaitons donc réaliser un produit matriciel et mettre le résultat dans une autre matrice.

Une version naïve peut être

In [14]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

Initialisons le problème afin de tester notre kernel.

In [15]:
n = 1024
A = np.random.rand(n, n)
B = np.random.rand(n, n)

d_A = cuda.to_device(A)
d_B = cuda.to_device(B)

d_C = cuda.device_array_like(A)

TPB = 16
BPG = (n + TPB - 1) // TPB

Nous procédons de la même manière que précédemment pour obtenir le temps de calcul.

In [16]:
%timeit matmul[(BPG, BPG), (TPB, TPB)](d_A, d_B, d_C)

The slowest run took 448.30 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 812 µs per loop


Vous devriez constater que c'est très rapide (peut-être un peu trop rapide ...).

#### Calcul des performances

Lorsque l'on calcule les performances d'un kernel GPU, il faut être très vigilant. Lorsque vous appelez votre fonction, Numba vous rend tout de suite la main pour pouvoir continuer à faire des calculs sur le CPU (ce qui permet d'avoir un recouvrement des calculs entre le CPU et le GPU). Donc tant que vous ne cherchez pas à récupérer les données de la carte, le calcul se fait en arrière plan sans synchronisation.

Afin de calculer correctement le temps de calcul, il est nécessaire de demander une synchronisation qui nous dit que tant que le calcul sur le GPU n'est pas terminé, nous n'allons pas plus loin.

In [17]:
%timeit matmul[(BPG, BPG), (TPB, TPB)](d_A, d_B, d_C); cuda.synchronize()

10 loops, best of 3: 121 ms per loop


In [18]:
np.allclose(d_C.copy_to_host(), A@B)

True

Voilà qui semble plus raisonnable. Nous allons à présent voir comment optimiser ce kernel. La démarche est souvent la même. Il est coûteux d'aller chercher les données sur la mémoire globale. Or lors de la mutlplication, nous allons souvent chercher les mêmes valeurs des matrices A et B. 

#### Mémoire partagée

Les cartes GPU sont dotées de mémoires partagées accessibles par l'ensemble des threads d'un même bloc. Numba permet d'allouer de la mémoire dans cet espace via

In [ ]:
cuda.shared.array(shape, dtype)

Le produit matrice-matrice peut donc être découpé en deux étapes

- l'ensemble des threads d'un même bloc met les blocs des matrices A et B dans la mémoire partagée
- on fait le calcul permettant de calculer un bout de C

La figure suivante résume la procédure

![matmult](figures/matrix-multiplication-with-shared-memory.png)

#### Synchronisation

Il est très important de remarquer que la deuxième phase ne peut se faire que lorsque l'ensemble des threads ont écrit leur partie dans le tableau partagé. Il est donc nécessaire de s'assurer que le tableau est prêt. Pour se faire, nous avons besoin de faire une synchronisation indiquant à l'ensemble des threads d'attendre temps que tout le monde n'est pas là. Pour se faire, on utilisera la fonction Numba suivante

In [ ]:
cuda.syncthreads()

Le kernel optimisé peut s'écrire

In [21]:
from numba import float64

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float64)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float64)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]
            
        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

**Attention** : ce kernel ne fonctionne que si n est un multiple du nombre de threads.

In [22]:
%timeit fast_matmul[(BPG, BPG), (TPB, TPB)](d_A, d_B, d_C); cuda.synchronize()

1 loop, best of 3: 261 ms per loop


In [23]:
np.allclose(d_C.copy_to_host(), A@B)

True

#### Fonctions sur le device

Il est également possible de créer des fonctions qui sont appelables uniquement sur le device. 

Par exemple

In [7]:
import math

@cuda.jit(device=True)
def sincos(a, b):
    return math.sin(a)*math.cos(b)

@cuda.jit
def test_device_func(a, b, c):
    i, j = cuda.grid(2)
    
    if i < c.shape[0] and j < c.shape[1]:
        c[i, j] = sincos(a[i], b[j])

In [8]:
a = np.linspace(0, 2*np.pi, 1000)
b = np.linspace(0, 2*np.pi, 1000)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.device_array((1000, 1000))

TPB = 32
BPG = (1000 + TPB - 1) // TPB

In [9]:
test_device_func[(BPG, BPG), (TPB, TPB)](d_a, d_b, d_c)

In [15]:
np.allclose(d_c.copy_to_host(), np.sin(a[:, np.newaxis])*np.cos(b))

True

## Exercices

#### Exercice 1

Implémentez sur GPU en utilisant **@cuda**, la fonction suivante avec et sans l'utilisation de la mémoire partagée

In [ ]:
def splint(xa, ya, y2a, x, y):
    n = xa.shape[0]
    for i in range(x.shape[0]):
        klo = 0
        khi = n-1
        while(khi-klo) > 1:
            k = (khi+klo) >> 1
            if xa[k] > x[i]:
                khi = k
            else:
                klo = k
        h = xa[khi] - xa[klo]
        a = (xa[khi]-x[i])/h    
        b = (x[i]-xa[klo])/h
        y[i,:] = a*ya[klo,:]+b*ya[khi,:]+((a**3-a)*y2a[klo,:]+(b**3-b)*y2a[khi,:])*h**2/6.

#### Exercice 2
Calculez les performances et observez à partir de quand cela devient intéressant.

In [1]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("./style/custom.css").read()
    return HTML(styles)
css_styling()